#### Установка зависимостей

In [ ]:
!pip install transformers accelerate
!huggingface-cli login
!pip install --upgrade pip
!pip install --force-reinstall "unsloth[cu121-torch240] @ git+https://github.com/unslothai/unsloth.git"
!pip install --force-reinstall unsloth_zoo

In [ ]:
!pip install optuna

#### Model

In [ ]:
import unsloth
from unsloth import FastLanguageModel
from transformers import TextStreamer
import torch

MODEL_NAME = "defog/sqlcoder-7b"

# загрузка модели
model_sql, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=2048,
    load_in_4bit=True
)

#### DataSet

In [ ]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="queries_marked_602_915.jsonl")
print(dataset)

full_dataset = dataset["train"]


temp_split = full_dataset.train_test_split(test_size=0.2, seed=42, shuffle=True)
train_dataset = temp_split["train"]
temp = temp_split["test"]

val_test = temp.train_test_split(test_size=0.5, seed=42, shuffle=True)

eval_dataset = val_test["train"]
test_dataset = val_test["test"]
print(train_dataset[0])
print(len(train_dataset), len(eval_dataset), len(test_dataset))

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(example):
    prompt = example["instruction"]
    if example["input"].strip() != "":
        prompt += "\nInput: " + example["input"]
    prompt += "\nOutput: " + example["output"]
    return tokenizer(prompt, truncation=True, padding="max_length", max_length=512)

tokenized_train = train_dataset.map(tokenize_function, batched=False)
tokenized_eval = eval_dataset.map(tokenize_function, batched=False)


#### Optuna

In [ ]:
import optuna
from transformers import TrainingArguments
from trl import SFTTrainer
import torch
from unsloth import is_bfloat16_supported

def objective(trial):
    # LoRA параметры
    r = trial.suggest_int("lora_r", 4, 64)
    lora_alpha = trial.suggest_int("lora_alpha", 16, 512)
    lora_dropout = trial.suggest_float("lora_dropout", 0.0, 0.4)
    use_rslora = trial.suggest_categorical("use_rslora", [True, False])
    bias_choice = trial.suggest_categorical("bias", ["none", "lora_only", "all"])
    target_modules_choice = trial.suggest_categorical(
        "target_modules",
        [
            ["q_proj", "k_proj", "v_proj", "o_proj"],
            ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
            ["q_proj", "k_proj", "v_proj"],
        ]
    )
    loftq_choice = trial.suggest_categorical("loftq_config", [None, {"bits": 4}, {"bits": 8}])


    learning_rate = trial.suggest_float("learning_rate", 1e-6, 5e-4, log=True)
    per_device_train_batch_size = trial.suggest_categorical("per_device_train_batch_size", [2, 4, 8, 16])
    per_device_eval_batch_size = per_device_train_batch_size
    gradient_accumulation_steps = trial.suggest_categorical("gradient_accumulation_steps", [1, 2, 4, 8, 16])
    weight_decay = trial.suggest_float("weight_decay", 0.0, 0.1)
    num_train_epochs = trial.suggest_int("num_train_epochs", 1, 5)
    lr_scheduler_type = trial.suggest_categorical(
        "lr_scheduler_type", ["linear", "cosine", "cosine_with_restarts", "polynomial"]
    )
    warmup_ratio = trial.suggest_float("warmup_ratio", 0.0, 0.3)
    warmup_steps = trial.suggest_int("warmup_steps", 0, 200)
    optim_choice = trial.suggest_categorical(
        "optim", ["adamw_torch_fused", "adamw_torch", "adamw_bnb_8bit", "paged_adamw_32bit"]
    )

    # Adam параметры
    adam_beta1 = trial.suggest_float("adam_beta1", 0.8, 0.95)
    adam_beta2 = trial.suggest_float("adam_beta2", 0.95, 0.999)
    adam_epsilon = trial.suggest_float("adam_epsilon", 1e-9, 1e-7, log=True)

    # Стабильность
    max_grad_norm = trial.suggest_float("max_grad_norm", 0.5, 5.0)
    gradient_checkpointing = trial.suggest_categorical("gradient_checkpointing", [True, False])
    seed_choice = trial.suggest_int("seed", 1, 9999)
    dataloader_num_workers = trial.suggest_int("dataloader_num_workers", 0, 4)

    # Выбор FP16/BF16
    fp16_option = trial.suggest_categorical("fp16", [True, False])
    bf16_option = trial.suggest_categorical("bf16", [True, False])
    if bf16_option and not is_bfloat16_supported():
        bf16_option = False
    if bf16_option:
        fp16_option = False

    # Data
    max_seq_length = trial.suggest_categorical("max_seq_length", [256, 512, 1024])
    packing_choice = trial.suggest_categorical("packing", [True, False])

    # Создаём LoRA модель
    model_lora = FastLanguageModel.get_peft_model(
        model_sql,
        r=r,
        target_modules=target_modules_choice,
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        bias=bias_choice,
        use_gradient_checkpointing=gradient_checkpointing,
        random_state=seed_choice,
        use_rslora=use_rslora,
        loftq_config=loftq_choice,
    )

    # TrainingArguments
    args = TrainingArguments(
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=per_device_eval_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        lr_scheduler_type=lr_scheduler_type,
        warmup_ratio=warmup_ratio,
        warmup_steps=warmup_steps,
        optim=optim_choice,
        adam_beta1=adam_beta1,
        adam_beta2=adam_beta2,
        adam_epsilon=adam_epsilon,
        max_grad_norm=max_grad_norm,
        logging_steps=trial.suggest_int("logging_steps", 10, 100),
        eval_steps=25,
        save_strategy=trial.suggest_categorical("save_strategy", ["no", "steps", "epoch"]),
        save_steps=trial.suggest_int("save_steps", 50, 200),
        fp16=fp16_option,
        bf16=bf16_option,
        output_dir="/tmp/optuna_trial",
        seed=seed_choice,
        dataloader_num_workers=dataloader_num_workers,
    )

    # Trainer
    trainer = SFTTrainer(
        model=model_lora,
        tokenizer=tokenizer,
        train_dataset=tokenized_train.shuffle(seed=42).select(range(100)),  # быстрый тест
        eval_dataset=tokenized_eval.shuffle(seed=42).select(range(20)),
        max_seq_length=max_seq_length,
        packing=packing_choice,
        args=args,
    )

    trainer.train()
    eval_results = trainer.evaluate()

    # pruning signal (чтобы Optuna мог прервать неудачные trial)
    trial.report(eval_results["eval_loss"], step=0)
    if trial.should_prune():
        raise optuna.TrialPruned()

    return eval_results["eval_loss"]



study = optuna.create_study(
    direction="minimize",
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=50)
)
study.optimize(objective, n_trials=50)

print("Лучшие гиперпараметры:", study.best_params)
